# Basit NLP Uygulamaları

In [1]:
import nltk
import textblob
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob, Word
from nltk import word_tokenize, pos_tag, ne_chunk

nltk.download("punkt", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("averaged_perceptron_tagger", quiet=True)

True

In [2]:
a = """Bu örneği anlaşılabilmesi için daha uzun bir metin üzerinden göstereceğim. N-Gram'lar birlikte kullanılan kelimelerin kombinasyonlarını gösterir."""
a

"Bu örneği anlaşılabilmesi için daha uzun bir metin üzerinden göstereceğim. N-Gram'lar birlikte kullanılan kelimelerin kombinasyonlarını gösterir."

## 1. N-Gram

In [3]:
TextBlob(a).ngrams(1)

[WordList(['Bu']),
 WordList(['örneği']),
 WordList(['anlaşılabilmesi']),
 WordList(['için']),
 WordList(['daha']),
 WordList(['uzun']),
 WordList(['bir']),
 WordList(['metin']),
 WordList(['üzerinden']),
 WordList(['göstereceğim']),
 WordList(["N-Gram'lar"]),
 WordList(['birlikte']),
 WordList(['kullanılan']),
 WordList(['kelimelerin']),
 WordList(['kombinasyonlarını']),
 WordList(['gösterir'])]

In [4]:
TextBlob(a).ngrams(2)

[WordList(['Bu', 'örneği']),
 WordList(['örneği', 'anlaşılabilmesi']),
 WordList(['anlaşılabilmesi', 'için']),
 WordList(['için', 'daha']),
 WordList(['daha', 'uzun']),
 WordList(['uzun', 'bir']),
 WordList(['bir', 'metin']),
 WordList(['metin', 'üzerinden']),
 WordList(['üzerinden', 'göstereceğim']),
 WordList(['göstereceğim', "N-Gram'lar"]),
 WordList(["N-Gram'lar", 'birlikte']),
 WordList(['birlikte', 'kullanılan']),
 WordList(['kullanılan', 'kelimelerin']),
 WordList(['kelimelerin', 'kombinasyonlarını']),
 WordList(['kombinasyonlarını', 'gösterir'])]

In [5]:
TextBlob(a).ngrams(3)

[WordList(['Bu', 'örneği', 'anlaşılabilmesi']),
 WordList(['örneği', 'anlaşılabilmesi', 'için']),
 WordList(['anlaşılabilmesi', 'için', 'daha']),
 WordList(['için', 'daha', 'uzun']),
 WordList(['daha', 'uzun', 'bir']),
 WordList(['uzun', 'bir', 'metin']),
 WordList(['bir', 'metin', 'üzerinden']),
 WordList(['metin', 'üzerinden', 'göstereceğim']),
 WordList(['üzerinden', 'göstereceğim', "N-Gram'lar"]),
 WordList(['göstereceğim', "N-Gram'lar", 'birlikte']),
 WordList(["N-Gram'lar", 'birlikte', 'kullanılan']),
 WordList(['birlikte', 'kullanılan', 'kelimelerin']),
 WordList(['kullanılan', 'kelimelerin', 'kombinasyonlarını']),
 WordList(['kelimelerin', 'kombinasyonlarını', 'gösterir'])]

## 2. Part of Speech Tagging (POS)
* Metindeki bulunan kelimelerin; sıfat, zarf, isim olma durumlarını tespit etmek için kullanılıyor.
* Önceki bölümde yaptıklarımızı buraya geçiriyoruz.

In [6]:
metin = """
A Scandal in Bohemia! 01
The Red-headed League,2
A Case, of Identity 33
The Boscombe Valley Mystery4
The Five Orange Pips1
The Man with? the Twisted Lip
The Adventure of the Blue Carbuncle
The Adventure of the Speckled Band
The Adventure of the Engineer's Thumb
The Adventure of the Noble Bachelor
The Adventure of the Beryl Coronet
The Adventure of the Copper Beeches"""

v_metin = metin.split("\n")
v_metin = pd.Series(v_metin)

metin_vektoru = v_metin[1:len(v_metin)]

mdf = pd.DataFrame(metin_vektoru, columns = ["Hikayeler"])
mdf["Hikayeler"] = mdf["Hikayeler"].apply(lambda x: " ".join(x.lower() for x in x.split()))
mdf["Hikayeler"] = mdf["Hikayeler"].str.replace("[^\w\s]","")
mdf["Hikayeler"] = mdf["Hikayeler"].str.replace("\d","")

sw = stopwords.words("english")

mdf["Hikayeler"] = mdf["Hikayeler"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
mdf

,Hikayeler
1,scandal bohemia
2,redheaded league
3,case identity
4,boscombe valley mystery
5,five orange pips
6,man twisted lip
7,adventure blue carbuncle
8,adventure speckled band
9,adventure engineers thumb
10,adventure noble bachelor


In [7]:
TextBlob(mdf["Hikayeler"][2]).tags

[('redheaded', 'VBN'), ('league', 'NN')]

In [8]:
mdf["Hikayeler"].apply(lambda x: TextBlob(x).tags)

1                        [(scandal, NN), (bohemia, NN)]
2                      [(redheaded, VBN), (league, NN)]
3                          [(case, NN), (identity, NN)]
4         [(boscombe, NN), (valley, NN), (mystery, NN)]
5               [(five, CD), (orange, NN), (pips, NNS)]
6                [(man, NN), (twisted, VBD), (lip, NN)]
7        [(adventure, NN), (blue, JJ), (carbuncle, NN)]
8        [(adventure, NN), (speckled, VBD), (band, NN)]
9     [(adventure, NN), (engineers, NNS), (thumb, VBP)]
10       [(adventure, NN), (noble, JJ), (bachelor, NN)]
11        [(adventure, NN), (beryl, NN), (coronet, NN)]
12      [(adventure, NN), (copper, NN), (beeches, NNS)]
Name: Hikayeler, dtype: object

## 3. Chunking (Shallow Parsing)
* Bir öncekini diyagram ile göstermektir.

In [9]:
cumle = "R and Python are useful data science tools for the new or old data scientists who eager to do efficent data science task"
cumle

'R and Python are useful data science tools for the new or old data scientists who eager to do efficent data science task'

In [10]:
pos = TextBlob(cumle).tags
pos

[('R', 'NNP'),
 ('and', 'CC'),
 ('Python', 'NNP'),
 ('are', 'VBP'),
 ('useful', 'JJ'),
 ('data', 'NNS'),
 ('science', 'NN'),
 ('tools', 'NNS'),
 ('for', 'IN'),
 ('the', 'DT'),
 ('new', 'JJ'),
 ('or', 'CC'),
 ('old', 'JJ'),
 ('data', 'NNS'),
 ('scientists', 'NNS'),
 ('who', 'WP'),
 ('eager', 'VBP'),
 ('to', 'TO'),
 ('do', 'VB'),
 ('efficent', 'JJ'),
 ('data', 'NNS'),
 ('science', 'NN'),
 ('task', 'NN')]

In [ ]:
reg_exp = "NP: {<DT>?<JJ>*<NN>}"
rp = nltk.RegexpParser(reg_exp)
sonuclar = rp.parse(pos)
sonuclar.draw()

## 4. Named Entity Recognition
* Verilen metinlerin içerisindeki öncekine benzer şekilde tanımlama işlemleri gerçekleştirir. Kelimelerin ne olduğu tanımlamalarını yapmak istersek kullanabiliriz.

In [ ]:
nltk.download("maxent_ne_chunker", quiet=True)
nltk.download("words", quiet=True)

In [ ]:
cumle = "Hadley is creative people who work for R Studio AND he attented conference at Newyork last year"
print(ne_chunk(pos_tag(word_tokenize(cumle))))